In [10]:
import keras
from keras.applications.vgg19 import VGG19
from keras.preprocessing import image
from keras.applications.vgg19 import preprocess_input
from keras.layers import Input, Flatten, Dense
from keras.models import Model
import numpy as np
from keras.initializers import Zeros
from keras.models import model_from_json
from keras.layers import Input, Concatenate, Conv2D, MaxPooling2D, Conv2DTranspose
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D, Conv2D
from keras.layers import Dense, Dropout, Activation, Flatten, Reshape, BatchNormalization, \
    Convolution2D, MaxPooling2D, ZeroPadding2D, Input, Embedding, LSTM, merge, Conv2DTranspose, \
    Lambda, UpSampling2D, Deconvolution2D, Cropping2D

In [11]:
#config = open("Keras_model_structure.json", "rb").read()
#json_model = model_from_json(config)

In [14]:
#Get back the convolutional part of a VGG network trained on ImageNet
model_vgg19_conv = VGG19(weights='imagenet', include_top=False)
model_vgg19_conv.summary()
#Create your own input format (here 256x256x3)
input = Input(shape=(256,256,3),name = 'image_input')

#Use the generated model 
output_vgg19_conv = model_vgg19_conv(input)

# y = model_vgg19_conv.layers[-2].output

concat = Input(shape=(256,256,3), name = 'concat')

#Add the fully-connected layers 
# x = Flatten(name='flatten')(output_vgg19_conv)

# x = Reshape(target_shape=(128,128,2))(output_vgg19_conv)
# upsample2_zeropadding = ZeroPadding2D((1,1),(1,1), name= "upsample2_zeropadding", data_format="channels_last")(model_vgg19_conv.layers[-17].output)

conv2_3_16_zeropadding = ZeroPadding2D(padding=(1,1), name= "conv2_3_16_zeropadding", data_format="channels_last")(model_vgg19_conv.layers[-17].output)

conv3_3_16_zeropadding = ZeroPadding2D(padding=(1,1), name= "conv3_3_16_zeropadding", data_format="channels_last")(model_vgg19_conv.layers[-12].output)

conv4_3_16_zeropadding = ZeroPadding2D(padding=(1,1), name= "conv4_3_16_zeropadding", data_format="channels_last")(model_vgg19_conv.layers[-7].output)

conv5_3_16_zeropadding = ZeroPadding2D(padding=(1,1), name= "conv5_3_16_zeropadding", data_format="channels_last")(model_vgg19_conv.layers[-2].output)



conv2_3_16 = Conv2D(filters=16, kernel_size=(3, 3), strides=(1, 1), dilation_rate=(1,1), activation='linear', use_bias=True, bias_initializer=Zeros(), name= "conv2_3_16", data_format="channels_last")(conv2_3_16_zeropadding)

conv3_3_16 = Conv2D(filters=16, kernel_size=(3, 3), strides=(1, 1), dilation_rate=(1,1), activation='linear', use_bias=True, bias_initializer=Zeros(), name= "conv3_3_16", data_format="channels_last")(conv3_3_16_zeropadding)

conv4_3_16 = Conv2D(filters=16, kernel_size=(3, 3), strides=(1, 1), dilation_rate=(1,1), activation='linear', use_bias=True, bias_initializer=Zeros(), name= "conv4_3_16", data_format="channels_last")(conv4_3_16_zeropadding)

conv5_3_16 = Conv2D(filters=16, kernel_size=(3, 3), strides=(1, 1), dilation_rate=(1,1), activation='linear', use_bias=True, bias_initializer=Zeros(), name= "conv5_3_16", data_format="channels_last")(conv5_3_16_zeropadding)


upsample2_zeropadding = ZeroPadding2D(padding=(1,1), name= "upsample2_zeropadding", data_format="channels_last")(conv2_3_16)


new_score_weighting = Conv2D(filters=1, kernel_size=(3, 3), strides=(1, 1), dilation_rate=(1,1), activation='linear', use_bias=True, bias_initializer=Zeros(), name= "new_score_weighting", data_format="channels_last")(concat)


upsample2_ = Conv2DTranspose(filters=16, kernel_size=(4, 4), strides=(2, 2), activation='linear', use_bias=True, bias_initializer=Zeros(), name= "upsample2_", data_format="channels_last")(upsample2_zeropadding)

upsample4_ = Conv2DTranspose(filters=16, kernel_size=(8, 8), strides=(4, 4), activation='linear', use_bias=True, bias_initializer=Zeros(), name= "upsample4_", data_format="channels_last")(conv3_3_16)

upsample8_ = Conv2DTranspose(filters=16, kernel_size=(16, 16), strides=(8, 8), activation='linear', use_bias=True, bias_initializer=Zeros(), name= "upsample8_", data_format="channels_last")(conv4_3_16)

upsample16_ = Conv2DTranspose(filters=16, kernel_size=(32, 32), strides=(16, 16), activation='linear', use_bias=True, bias_initializer=Zeros(), name= "upsample16_", data_format="channels_last")(conv5_3_16)


sigmoid_fuse = Activation(activation="sigmoid")(new_score_weighting)

#x = json_model(x)


#Create your own model 
my_model = Model(input=[model_vgg19_conv.layers[-22].output, concat] , output=[upsample2_, upsample4_, upsample8_, upsample16_, sigmoid_fuse])

#In the summary, weights and layers from VGG part will be hidden, but they will be fit during the training
my_model.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_8 (InputLayer)         (None, None, None, 3)     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, None, None, 128)   0         
__________

/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/ipykernel_launcher.py:60: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=[<tf.Tenso..., inputs=[<tf.Tenso...)`
